In [1]:
import pandas as pd
import sqlite3

In [2]:
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q,conn)

In [3]:
def run_command(my_comm):
    conn.isolation_level = None
    conn.execute(my_comm)
    

In [4]:
def show_tables():
    show_all = ''' 
    SELECT name, type 
    FROM sqlite_master 
    WHERE type IN ("table","view")
    '''
    return run_query(show_all)

In [5]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


In [6]:
tracks_usa = '''
WITH tracks_usa AS (
        SELECT il.track_id,
              SUM(quantity) quantity 
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        GROUP BY track_id
        HAVING billing_country = "USA"),

    genre_name AS (
        SELECT t.track_id, g.name
        FROM track t
        INNER JOIN genre g ON g.genre_id = t.genre_id)
        
SELECT gn.name genre, 
       SUM(tu.quantity) quantity, 
       ROUND(CAST(SUM(tu.quantity) AS FLOAT)*100/1149, 3) percentage
FROM genre_name gn 
INNER JOIN tracks_usa tu ON tu.track_id = gn.track_id
GROUP BY genre

'''
run_query(tracks_usa)
genre_sales_usa = run_query(tracks_usa)

From the data above, we can see that Rock tracks are usually the most sold in USA. This constitutes over 50% of track sales. Metal as well as Alternative & Punk are also popular. 

Based on this information, the following artists' albums should be purchased for the store: 
- Red Tone (Punk) 
- Regal (Hip-Hop) 
- Meteor and the Girls (Pop)


In [7]:
total_sales_q = '''
WITH agent AS (SELECT first_name || " " || last_name employee, 
                      employee_id, hire_date
               FROM employee e
               WHERE title IS "Sales Support Agent"),
     sales AS (SELECT c.support_rep_id, SUM(i.total) total
               FROM customer c 
               INNER JOIN invoice i ON c.customer_id = i.customer_id
               GROUP BY support_rep_id)
SELECT employee, total, hire_date
FROM agent a
INNER JOIN sales s ON s.support_rep_id = a.employee_id      
'''

run_query(total_sales_q)

,employee,total,hire_date
0,Jane Peacock,1731.51,2017-04-01 00:00:00
1,Margaret Park,1584.00,2017-05-03 00:00:00
2,Steve Johnson,1393.92,2017-10-17 00:00:00


Jane Peacock has the highest dollar amount of sales in the company and she is also has the longest employment history among the Sales Support Agents. 

Although Steve Johnson sales are the least, he has considerably spent at least 5 months less than his colleagues in the role.

In [8]:
run_query('''
WITH amount AS (
        SELECT
          CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
        END AS country, c.customer_id, quantity, total 
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id)
     
SELECT country,  
           SUM(total) total,
           CAST(COUNT(country) AS FLOAT) customers,
           SUM(quantity) quantity,
           total/quantity AS average_order,
           total/COUNT(country) average_sales_per_customer
FROM amount
GROUP BY country
''')


,country,total,customers,quantity,average_order,average_sales_per_customer
0,Brazil,4059.00,432.0,432,6.93,0.016042
1,Canada,5489.55,541.0,541,11.88,0.021959
2,Czech Republic,3183.84,276.0,276,0.99,0.003587
3,France,3972.87,393.0,393,5.94,0.015115
4,Germany,3441.24,338.0,338,7.92,0.023432
5,India,1887.93,185.0,185,4.95,0.026757
6,Other,10808.82,1106.0,1106,6.93,0.006266
7,Portugal,1755.27,187.0,187,3.96,0.021176
8,USA,10405.89,1051.0,1051,8.91,0.008478
9,United Kingdom,2498.76,248.0,248,12.87,0.051895


In [15]:
run_query('''
SELECT t.*,il.* 
FROM track t 
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY invoice_id ASC
''')

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1158,Right Next Door to Hell,91,2,1,None,182321,3175950,0.99,1,1,1158,0.99,1
1,1159,Dust N' Bones,91,2,1,None,298374,5053742,0.99,2,1,1159,0.99,1
2,1160,Live and Let Die,91,2,1,None,184016,3203390,0.99,3,1,1160,0.99,1
3,1161,Don't Cry (Original),91,2,1,None,284744,4833259,0.99,4,1,1161,0.99,1
4,1162,Perfect Crime,91,2,1,None,143637,2550030,0.99,5,1,1162,0.99,1
5,1163,You Ain't the First,91,2,1,None,156268,2754414,0.99,6,1,1163,0.99,1
6,1164,Bad Obsession,91,2,1,None,328282,5537678,0.99,7,1,1164,0.99,1
7,1165,Back off Bitch,91,2,1,None,303436,5135662,0.99,8,1,1165,0.99,1
8,1166,Double Talkin' Jive,91,2,1,None,203637,3520862,0.99,9,1,1166,0.99,1
9,1167,November Rain,91,2,1,None,537540,8923566,0.99,10,1,1167,0.99,1


In [18]:
run_query('''
SELECT a.*,t.* 
FROM track t 
INNER JOIN album a ON a.album_id = t.album_id
ORDER BY album_id ASC
LIMIT 30
''')

,album_id,title,artist_id,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,1,For Those About To Rock We Salute You,1,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,1,For Those About To Rock We Salute You,1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,1,For Those About To Rock We Salute You,1,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,1,For Those About To Rock We Salute You,1,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,1,For Those About To Rock We Salute You,1,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
5,1,For Those About To Rock We Salute You,1,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99
6,1,For Those About To Rock We Salute You,1,11,C.O.D.,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",199836,6566314,0.99
7,1,For Those About To Rock We Salute You,1,12,Breaking The Rules,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,0.99
8,1,For Those About To Rock We Salute You,1,13,Night Of The Long Knives,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205688,6706347,0.99
9,1,For Those About To Rock We Salute You,1,14,Spellbound,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",270863,8817038,0.99
